# Cats vs. Dogs Redux: Kernels Edition

A notebook for the Cats vs. Dogs Kaggle competition (found at https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/data). I'll borrow some pre-processing steps from various other kernels, but build my own Tensorflow neural network.

## Imports

In [2]:
import numpy as np
import pandas as np
import tensorflow as tf
import matplotlib.pyplot as plt
import PIL

%matplotlib inline

## Data Pre-processing

It's common in image analysis to normalize luminance (brightness) values to have mean 0 and standard deviation 1. We do that, and apply a slight contrast stretch which ensures that brightness values stay within the bounds of the image encoding.

The normalization is applied to luminance (not RBG channels individually), so we first convert to YCbCr space, operate on the Y channel, and then convert back to RBG color channels.

In [4]:
def norm_img(img):
    '''
    Normalize a PIL image. This normalizes luminance to have mean 0 and standard deviation 1, and applies a
    [1%, 99%] contrast stretch.
    '''
    # convert to YCbCr and split into y, b, r channels
    img_y, img_b, img_r = img.convert('YCbCr').split()
    
    # convert the y channel to a numpy array representtaion
    img_y = np.asarray(img_y).astype(float)
    
    # normalize in range [0, 1], zero-mean, and unit standard deviation
    img_y /= 255
    img_y -= img_y.mean()
    img_y /= img_y.std()
    
    # apply contrast stretch
    scale = np.max([np.abs(np.percentile(img_y, 1.0)), np.abs(np.percentile(img_y, 99.0))])
    img_y /= scale
    img_y = np.clip(img_y, -1.0, 1.0)
    img_y = (img_y + 1.0) / 2.0
    
    # rescale back to range [0, 255]
    img_y = (img_y * 255 + 0.5).astype(np.uint8)
    
    # create PIL image from img_y
    img_y = PIL.Image(img_y)
    
    # merge the y channel image back with the others and convert back to RGB
    img_nrm = PIL.Image.merge('YCbCr', (img_y, img_b, img_r)).convert('RGB')
    
    # return the luminace normalized image
    return img_nrm

## Resize Images

We resize the images to be square with default side lengths of 224 pixels. The aspect ratio is preserved and gray bars are added as needed to make the image square.

In [5]:
def resize_img(img, size):
    '''
    Resize a PIL image to be square with side length 'size', and pads with gray bars if necessary.
    '''
    # get image's original size
    width, height = img.size
    
    # make larger side length equal to 'size' and maintain aspect ratio
    if height > width:
        height, width = size, int(size * width / height + 0.5)
    else:
        width, height = size, int(size * hight / width + 0.5)
    
    # reshaping...
    img_res = img.resize((width, height), resample=PIL.Image.BICUBIC)
    
    # pad the borders to create a square image
    img_pad = PIL.Image.new('RGB', (size, size), (128, 128, 128))
    ulc = ((size - width) // 2, (size - height) // 2)
    img_pad.paste(img_res, ulc)
    
    # return the padded, resized image
    return img_pad